In [4]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import models

from ssdmultibox.datasets import TrainPascalDataset
from ssdmultibox.model import CustomHead

In [5]:
model = models.resnet18(pretrained=True)
model.fc = CustomHead()

# data
dataset = TrainPascalDataset(grid_size=4)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# process
item = next(iter(dataloader))
image_ids, ims, bbs, cats = item

# zero the parameter gradients
optimizer.zero_grad()

# forward pass
outputs = model(ims)

In [13]:
pred_bbs, pred_cats = outputs

In [14]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [15]:
target.shape

torch.Size([3])

In [19]:
target

tensor([3, 1, 4])

In [20]:
pred_cats.dtype

torch.float32

In [21]:
cats.dtype

torch.float64

In [24]:
pred_cats = torch.tensor(pred_cats, dtype=torch.float64)

In [25]:
bce = F.binary_cross_entropy_with_logits(pred_cats, cats)
bce

tensor(0.6976, dtype=torch.float64, grad_fn=<MeanBackward1>)